<a href="https://colab.research.google.com/github/AyaAhmed01/Fall-2022-Neural-Networks-Assignments/blob/main/A5-2_svhn_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Cairo University Faculty of Engineering
## Deep Learning
## Assignment 5_2

---

Please write your full name here
- **Name** : "Aya Emad FOuad Ahmed"

## Image classifier for the SVHN dataset
### Instructions

In this notebook, you will create a neural network that classifies real-world images digits.
Within this notebook you will find instructions in each section for how to complete the assignment.

Pay close attention to the instructions. Feel free to add extra cells into the notebook as required.

### Let's get started!

We'll start by running some imports, and loading the dataset. You are free to make further imports throughout the notebook as you wish.

For this assignmnet, you will use the [SVHN dataset](http://ufldl.stanford.edu/housenumbers/). This is an image dataset of over 600,000 digit images in all, and is a harder dataset than MNIST as the numbers appear in the context of natural scene images. SVHN is obtained from house numbers in Google Street View images.

* Y. Netzer, T. Wang, A. Coates, A. Bissacco, B. Wu and A. Y. Ng. "Reading Digits in Natural Images with Unsupervised Feature Learning". NIPS Workshop on Deep Learning and Unsupervised Feature Learning, 2011.

The train and test datasets required for this project can be downloaded from [here](http://ufldl.stanford.edu/housenumbers/train.tar.gz) and [here](http://ufldl.stanford.edu/housenumbers/test.tar.gz). Once unzipped, you will have two files: `train_32x32.mat` and `test_32x32.mat`.

- If using Colab: You should store these files in Drive.

Your goal is to develop an end-to-end workflow for building, training, validating, evaluating and saving a neural network that classifies a real-world image into one of ten classes.

In [ ]:
import tensorflow as tf
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense , BatchNormalization , Dropout ,Flatten
from tensorflow.keras import Sequential
import pandas as pd
%matplotlib inline

In [ ]:
# Run this cell to load the dataset

test = loadmat('/kaggle/input/test_32x32.mat')
train = loadmat('/kaggle/input/train_32x32.mat')

FileNotFoundError: ignored

Both `train` and `test` are dictionaries with keys `X` and `y` for the input images and labels respectively.

## 1. Inspect and preprocess the dataset
* Extract the training and testing images and labels separately from the train and test dictionaries loaded for you.
* Select a random sample of images and corresponding labels from the dataset (at least 10), and display them in a figure.
* Convert the training and test images to grayscale by taking the average across all colour channels for each pixel. _Hint: retain the channel dimension, which will now have size 1._
* Select a random sample of the grayscale images and corresponding labels from the dataset (at least 10), and display them in a figure.

In [ ]:
train_x = train['X']
train_y = train['y']
test_x = test['X']
test_y = test['y']

In [ ]:
train_y[train_y == 10] = 0
test_y[test_y == 10] = 0

In [ ]:
train_x.shape

(32, 32, 3, 73257)

In [ ]:
# Shape of training examples isn't the way that is usually dealt with so used np.transpose to permutate the axis
train_x = np.transpose(train_x,(3 ,0 ,1,2))
test_x = np.transpose(test_x,(3 ,0 ,1,2))

print("Train x shape : {}\nTest x shape : {}".format(train_x.shape,test_x.shape))

In [ ]:
# Normalize
train_x = train_x / 255
test_x = test_x / 255

In [ ]:
## ATTENTION: this's not my own function, it's grabbed from other notebook but I understood the code ##

def show_pictures_colored(indices,tset,tlabel):

    fig , axlist = plt.subplots(ncols = 5 , nrows = 2 )

    for ind , pind in enumerate(indices):

        axlist.ravel()[ind].set_axis_off()
        axlist.ravel()[ind].set_title('{}'.format(tlabel[pind]))
        axlist.ravel()[ind].imshow(tset[pind])
    fig.tight_layout()

def show_pictures_grey(indices,tset,tlabel):

    fig , axlist = plt.subplots(ncols = 5 , nrows = 2 )

    for ind , pind in enumerate(indices):

        axlist.ravel()[ind].set_axis_off()
        axlist.ravel()[ind].set_title('{}'.format(tlabel[pind]))
        axlist.ravel()[ind].imshow(tset[pind,:,:,0])
    fig.tight_layout()


In [ ]:
indices = list(np.random.randint(1,train_x.shape[0],size=(10,)))
show_pictures_colored(indices,train_x,train_y)

In [ ]:
train_x_grey = np.mean(train_x,keepdims=True,axis = -1)
test_x_grey = np.mean(test_x,keepdims=True,axis = -1)
train_x_grey.shape

In [ ]:
indices = list(np.random.randint(1,train_x.shape[0],size=(10,)) )
show_pictures_grey(indices , train_x_grey , train_y)

In [ ]:
# divide training into train & validation
from sklearn.model_selection import train_test_split

train_x , val_x , train_y , val_y = train_test_split(train_x_grey,train_y, test_size = 0.04 )

## 2. MLP neural network classifier
* Build an MLP classifier model using the Sequential API. Your model should use only Flatten and Dense layers, with the final layer having a 10-way softmax output.
* You should design and build the model yourself. Feel free to experiment with different MLP architectures. _Hint: to achieve a reasonable accuracy you won't need to use more than 4 or 5 layers._
* Print out the model summary (using the summary() method)
* Compile and train the model (we recommend a maximum of 30 epochs), making use of both training and validation sets during the training run.
* Your model should track at least one appropriate metric, and use at least two callbacks during training, one of which should be a ModelCheckpoint callback.
* As a guide, you should aim to achieve a final categorical cross entropy training loss of less than 1.0 (the validation loss might be higher).
* Plot the learning curves for loss vs epoch and accuracy vs epoch for both training and validation sets.
* Compute and display the loss and accuracy of the trained model on the test set.

In [ ]:
# Build model
def build_mlp():
    model = Sequential()
    model.add(Flatten(input_shape=(32,32)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
model = build_mlp()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

## Callbacks : save best and reducelr on plateau

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

callback_best = ModelCheckpoint('model/best_model' , save_best_only= True ,save_weights_only=True , monitor="val_accuracy" )
reduceLRplat = ReduceLROnPlateau(monitor="val_loss",patience=5)

In [ ]:
from keras.utils.np_utils import to_categorical

train_y_cat = to_categorical(train_y,10)    # Converts a class vector (integers) to binary class matrix (one hot encoding)
val_y_cat = to_categorical(val_y,10)

In [ ]:
history = model.fit(train_x, train_y_cat, validation_data= (val_x, val_y_cat), verbose= 2, epochs= 25,
                    batch_size= 300, callbacks= [callback_best, reduceLRplat])

In [ ]:
# list all data in history
# print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])      #dict_keys(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy', 'lr'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# df = pd.DataFrame(history.history())   # other way
# df.plot(y= ["loss","val_loss"],use_index = True)
# df.plot(y= [ "accuracy","val_accuracy"] , use_index = True)

In [ ]:
# Compute and display the loss and accuracy of the trained model on the test set.
[test_loss, test_acc] = model.evaluate(test_x_grey, to_categorical(test_y, 10))
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

## 3. CNN neural network classifier
* Build a CNN classifier model using the Sequential API. Your model should use the Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense and Dropout layers. The final layer should again have a 10-way softmax output.
* You should design and build the model yourself. Feel free to experiment with different CNN architectures. _Hint: to achieve a reasonable accuracy you won't need to use more than 2 or 3 convolutional layers and 2 fully connected layers.)_
* The CNN model should use fewer trainable parameters than your MLP model.
* Compile and train the model (we recommend a maximum of 30 epochs), making use of both training and validation sets during the training run.
* Your model should track at least one appropriate metric, and use at least two callbacks during training, one of which should be a ModelCheckpoint callback.
* You should aim to beat the MLP model performance with fewer parameters!
* Plot the learning curves for loss vs epoch and accuracy vs epoch for both training and validation sets.
* Compute and display the loss and accuracy of the trained model on the test set.

In [ ]:
# should use the Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense and Dropout layers.
# The final layer should again have a 10-way softmax output.
# 2 or 3 convolutional layers and 2 fully connected layers
# (30 epochs), making use of both training and validation sets during the training run.
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization

# a LeNet-5 like CNN
def build_cnn(x_train):
    model = Sequential([Conv2D(6,input_shape=(x_train[0].shape), strides=1, kernel_size= 5, activation= 'relu'),
                       MaxPooling2D(pool_size= 2, strides= 2),
                       Conv2D(16, strides=1, kernel_size= 5, activation= 'relu'),
                       MaxPooling2D(pool_size= 2, strides= 2),
                       BatchNormalization(),
                       Flatten(),
                       Dense(units= 120, activation= 'relu'),
                       Dropout(0.2),
                       Dense(units= 84, activation= 'relu'),
                       Dense(units= 10, activation= 'softmax')])
    return model

    #conv s=1 f=5, maxpooling f=s=2,  same conv,  same pooling, batch_norm,  flatten, fc 120, dropout, fc 84, softmax

In [ ]:
model = build_cnn(train_x)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping

callback_best_cnn = ModelCheckpoint('model_cnn/best_model' ,save_best_only=True ,  save_weights_only=True , monitor="val_accuracy" )

# Callback that streams epoch results to a CSV file.
# Supports all values that can be represented as a string, including 1D iterables such as np.ndarray.
csvlogger = CSVLogger('model_cnn.csv')

callback_earlyStopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
history = model.fit(train_x, train_y_cat, validation_data= (val_x, val_y_cat), verbose= 2, epochs= 20,
                    batch_size= 300, callbacks= [callback_best_cnn, csvlogger,callback_earlyStopping])

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['categorical_accuracy'])      #dict_keys(['loss', 'categorical_accuracy', 'val_loss', 'val_categorical_accuracy', 'lr'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Compute and display the loss and accuracy of the trained model on the test set.
[test_loss, test_acc] = model.evaluate(test_x_grey, to_categorical(test_y, 10))
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

## 4. Get model predictions
* Load the best weights for the MLP and CNN models that you saved during the training run.
* Randomly select 5 images and corresponding labels from the test set and display the images with their labels.
* Alongside the image and label, show each model’s predictive distribution as a bar chart, and the final model prediction given by the label with maximum probability.

In [ ]:
best_mlp = build_mlp()
best_mlp.load_weights('model/best_model')

In [ ]:
best_cnn = build_cnn()
best_cnn.load_weights('model_cnn/best_model')